In [1]:
import numpy as np
import timeit
from tqdm.auto import tqdm
import pandas as pd

def check_correctness(eigvals_func, A, rtol=1e-2, atol=1e-3):
    assert np.allclose(sorted(np.linalg.eigvals(A)), sorted(eigvals_func(A)), rtol=rtol, atol=atol)

def make_experiment(eigvals_func, N=50, random_seed=0, correctness_tests=10, rtol=1e-2, atol=1e-3, estimated_time=10, retries=7, filename=None):
    np.random.seed(random_seed)
    for _ in tqdm(range(correctness_tests), desc="Checking correctness"):
        A = np.random.randn(N, N)
        A = A.dot(A.T)
        check_correctness(eigvals_func, A, rtol=rtol, atol=atol)
        
    if filename is not None:
        with open(filename, "w") as f:
            f.write(str(N) + "\n")
            f.write("\n".join([" ".join(x) for x in A.astype(str)]))
        print(np.linalg.eigvals(A))
    
    variables = {'eigvals_func': eigvals_func, 'A': A}
    single_execution = timeit.timeit(stmt="eigvals_func(A)", globals=variables, number=1)
    number = max(int(np.floor(estimated_time / single_execution / retries)), 1)
    times = [
        timeit.timeit(stmt="eigvals_func(A)", globals=variables, number=number) / number for _ in range(retries)
    ]
    return times

make_experiment(np.linalg.eigvals)

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

[0.00043614168039538703,
 0.00046201070840197673,
 0.00044493196046128535,
 0.000409228006589786,
 0.00041123706754530486,
 0.00041419373970345976,
 0.00042207808896210824]

In [2]:
n_values = [5, 10, 20, 30, 40, 50]
results = []

for n in n_values:
    make_experiment(np.linalg.eigvals, N=n, filename=f"{n}.txt")

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

[12.54370076  0.06099317  0.32840781  3.21488977  5.07253904]


Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

[2.97847028e+01 2.55945678e+01 1.37579164e+01 1.11960774e+01
 5.88960347e+00 2.19202482e+00 1.54674224e+00 1.84117622e-03
 7.39450784e-01 3.73343632e-01]


Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

...

[9.00460824e+01 8.27476579e+01 7.81224383e+01 7.39693968e+01
 6.28985299e+01 5.46269232e+01 4.92271427e+01 4.73391856e+01
 4.61217258e+01 4.03773218e+01 3.22137523e+01 2.59248721e+01
 2.25558693e+01 2.14285986e+01 2.09775713e+01 1.90401177e+01
 1.78124993e+01 1.32351666e+01 1.05517238e+01 7.81905311e+00
 6.61877244e+00 3.84891455e+00 3.60692696e+00 2.69805863e+00
 5.42076630e-03 1.82603331e-01 2.47518564e-01 4.75412204e-01
 1.55740898e+00 1.12830437e+00]


Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

[1.39505693e+02 1.29782902e+02 1.07723769e+02 1.01323385e+02
 9.39111570e+01 8.56867569e+01 8.34660922e+01 7.80819178e+01
 7.00444239e+01 6.73257195e+01 5.86781762e+01 5.57201769e+01
 5.08377973e+01 4.78292344e+01 4.04295317e+01 3.73349174e+01
 3.56916216e+01 3.05912314e+01 2.83411950e+01 2.68548104e+01
 2.42034999e+01 2.21140187e+01 1.75930173e+01 1.49803437e+01
 1.37050212e+01 1.29064367e+01 9.98977672e+00 8.74418203e+00
 8.54349624e+00 6.22833462e+00 7.79032211e-03 3.34812420e-02
 5.61947750e-01 6.25508292e-01 1.67596452e+00 1.70278859e+00
 2.20408178e+00 3.33625130e+00 4.26344360e+00 4.04474026e+00]


Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

[2.08252232e+02 1.55036102e+02 1.51338704e+02 1.41693546e+02
 1.29723433e+02 1.16960667e+02 1.09974861e+02 1.01554156e+02
 1.00024539e+02 9.36810806e+01 9.12094337e+01 8.42505433e+01
 7.93069758e+01 7.53863463e+01 7.15386760e+01 6.84279459e+01
 6.86474914e+01 6.04217118e+01 5.00497457e+01 4.50241044e+01
 4.14457589e+01 3.81686158e+01 3.70437798e+01 3.43566358e+01
 3.16173522e+01 2.87320306e+01 2.53558473e+01 2.34377743e+01
 2.17943900e+01 1.98630919e+01 1.81079680e+01 1.73999698e+01
 1.56298249e+01 1.38775208e+01 1.20729259e+01 1.10012452e+01
 1.00839567e+01 8.07696803e+00 7.02445458e+00 5.95344559e+00
 5.65431829e+00 4.36454810e+00 3.24870662e+00 2.76939994e+00
 1.68527895e+00 9.05838023e-01 3.25387479e-03 4.19591503e-02
 1.39336256e-01 4.01936385e-01]


## Базовая имплементация на Numpy

In [3]:
def qr(A):
    Q = A.copy()
    Q[:, 0] /= np.linalg.norm(Q[:, 0])
    for i in range(1, A.shape[0]):
        Q[:, i] -= Q[:, :i].dot(Q[:, :i].T).dot(Q[:, i])
        Q[:, i] /= np.linalg.norm(Q[:, i])
    R = Q.T.dot(A)
    return Q, R

def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        Q, R = qr(A)
        A = R.dot(Q)
        squared_A = np.square(A)
    return np.diag(A)

for n in n_values:
    result = make_experiment(eigenvalues, N=n)
    results.append(result)

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Попытка ускорения итерации на Numpy

In [ ]:
def make_iteration(A):
    Q = A.copy()
    Q[:, 0] /= np.linalg.norm(Q[:, 0])
    for i in range(1, A.shape[0]):
        Q[:, i] -= Q[:, :i].dot(Q[:, :i].T).dot(Q[:, i])
        Q[:, i] /= np.linalg.norm(Q[:, i])
    return Q.T.dot(A).dot(Q)

def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        A = make_iteration(A)
        squared_A = np.square(A)
    return np.diag(A)

for n in n_values:
    result = make_experiment(eigenvalues, N=n)
    results.append(result)

## Оптимизация вычисления функции на Numpy за счёт кэширования результатов

In [ ]:
def make_iteration(A):
    Q = A.T.copy()
    saved_Q = np.zeros_like(A)
    for i in range(A.shape[0]):
        Q[i] -= saved_Q.dot(Q[i])
        Q[i] /= np.linalg.norm(Q[i])
        saved_Q += np.outer(Q[i], Q[i])
    return np.linalg.multi_dot([Q, A, Q.T])

def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        A = make_iteration(A)
        squared_A = np.square(A)
    return np.diag(A)

for n in n_values:
    result = make_experiment(eigenvalues, N=n)
    results.append(result)

## Numba JIT

In [ ]:
from numba import jit

@jit(nopython=True)
def make_iteration(A):
    Q = A.T.copy()
    saved_Q = np.zeros_like(A)
    for i in range(A.shape[0]):
        Q[i] -= saved_Q.dot(Q[i])
        Q[i] /= np.linalg.norm(Q[i])
        saved_Q += np.outer(Q[i], Q[i])
    return Q.dot(A).dot(Q.T)

@jit(nopython=True)
def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        A = make_iteration(A)
        squared_A = np.square(A)
    return np.diag(A)

for n in n_values:
    result = make_experiment(eigenvalues, N=n)
    results.append(result)

## Numba JIT с дополнительными параметрами

In [ ]:
from numba import jit

@jit(nopython=True, nogil=True, fastmath=True, cache=True)
def make_iteration(A):
    Q = A.T.copy()
    saved_Q = np.zeros_like(A)
    for i in range(A.shape[0]):
        Q[i] -= saved_Q.dot(Q[i])
        Q[i] /= np.linalg.norm(Q[i])
        saved_Q += np.outer(Q[i], Q[i])
    return Q.dot(A).dot(Q.T)

@jit(nopython=True, nogil=True, fastmath=True, cache=True)
def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        A = make_iteration(A)
        squared_A = np.square(A)
    return np.diag(A)

for n in n_values:
    result = make_experiment(eigenvalues, N=n)
    results.append(result)

## Numba guvectorize

In [ ]:
from numba import guvectorize, float64

@guvectorize(["void(float64[:, ::1], float64, float64, float64[::1])"], "(n,n), (), () -> (n)", nopython=True, target='cpu')
def eigenvalues(A, rtol=1e-6, atol=1e-8, out=None):
    squared_A = A * A
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        Q = A.T.copy()
        Q[0] /= np.linalg.norm(Q[0])
        saved_Q = np.zeros_like(A)
        for i in range(A.shape[0]):
            Q[i] -= saved_Q.dot(Q[i])
            Q[i] /= np.linalg.norm(Q[i])
            saved_Q += np.outer(Q[i], Q[i])
        A = Q.dot(A).dot(Q.T)
        
        squared_A = A * A
    out[:] = np.diag(A)

def eigenvalues_wrapper(A, rtol=1e-6, atol=1e-8):
    result = np.empty(A.shape[0], dtype=float)
    eigenvalues(A, rtol, atol, result)
    return result

for n in n_values:
    result = make_experiment(eigenvalues_wrapper, N=n)
    results.append(result)

In [ ]:
def f(x):
    return list(map(float, x.split()))

gg = [
    f("0.069969 0.080002 0.070009 0.069989 0.07 0.069998 0.08"),
    f("0.309973 0.300028 0.309973 0.32003 0.30997 0.310007 0.299993"),
    f("3.00883 3.13567 3.20012 3.14 3.17999 3.11904 3.16399"),
    f("21.4515 21.4018 21.6 21.5061 21.14 21.8804 21.641"),
    f("29.7156 29.3104 29.7019 31.9827 30.4195 29.6241 31.0762"),
    f("87.9677 87.8141 87.8033 87.9801 88.7885 87.4791 86.9187"),
]

In [ ]:
gg

In [ ]:
names = ['Numpy 1', 'Numpy 2', 'Numpy 3', 'Numba JIT', 'Numbda JIT с доп. пар.', 'Numba guvectorize', 'C++']

In [ ]:
total_results = np.array(results + gg).reshape(len(names), len(n_values), -1) * 1000

In [ ]:
R = pd.DataFrame(np.core.defchararray.add(np.core.defchararray.add(np.round(total_results.mean(axis=-1), 1).astype(str), " ± "), np.round(total_results.std(axis=-1), 1).astype(str)), index=names, columns=n_values)

In [ ]:
R

In [ ]:
for i, j in enumerate(np.round(total_results.mean(axis=-1), 1).argmin(axis=0)):
    R.iloc[j, i] = '\\textbf{' + R.iloc[j, i] + "}"

In [ ]:
R

In [ ]:
R[np.arange(10, 60, 10)].reset_index().to_csv("../../report/tables/results.csv", sep=';', index=False)